In [1]:
import numpy as np

data = np.load('../processed_data/split_dataset/data.npz', allow_pickle=True)
label = np.load('../processed_data/split_dataset/label.npz.npy')

In [2]:
X = np.stack([data[f'arr_{i}'].item()['three_sec'] for i in range(len(data))])
X_30sec = np.stack([data[f'arr_{i}'].item()['thirty_sec'] for i in range(len(data))])
label = label
RANDOM_STATE = 42

In [3]:
import numpy as np

def generate_k_fold_dataset(data_3sec, data_30sec, label, k, shrink_ratio=1.0, shuffle_train_data=True, random_state=42):
    np.random.seed(random_state)
    numbers = np.arange(data_3sec.shape[0])
    np.random.shuffle(numbers)
    test_indices = np.array_split(numbers, k)
    
    train_folds, test_3sec_folds, test_30sec_folds = [], [], []
    for i in range(k):
        mask = np.zeros(data_3sec.shape[0], dtype=bool)
        mask[test_indices[i]] = True
        
        X_train, y_train = data_3sec[~mask], label[~mask]
        if shuffle_train_data:
            shuffle_indices = np.arange(X_train.shape[0])
            np.random.shuffle(shuffle_indices)
            X_train, y_train = X_train[shuffle_indices], y_train[shuffle_indices]
        
        X_3sec_test, y_3sec_test = data_3sec[mask], label[mask]
        X_30sec_test, y_30sec_test = data_30sec[mask], label[mask]
        
        train_folds.append((X_train[:int(shrink_ratio * len(X_train))], y_train[:int(shrink_ratio * len(y_train))]))
        test_3sec_folds.append((X_3sec_test, y_3sec_test))
        test_30sec_folds.append((X_30sec_test, y_30sec_test))
    
    if shrink_ratio != 1.0:
        print(f'Warning: shrink_ratio is set to {shrink_ratio}, the dataset is shrinked.')
        print(f'X_train=({train_folds[0][0].shape}, y_train=({train_folds[0][1].shape}), X_3sec_test=({test_30sec_folds[0][0].shape}, y_3sec_test=({test_30sec_folds[0][1].shape})')
    return train_folds, test_3sec_folds, test_30sec_folds
        
train_folds, test_3sec_folds, test_30sec_folds = generate_k_fold_dataset(X, X_30sec, label, 10, shrink_ratio=1.0, shuffle_train_data=False)

In [4]:
from scipy import stats
from sklearn.svm import SVC


def predict_and_mode(model, X_test, scaler, threshold=0.6):
    X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    results = model.predict(X_test_scaled.reshape(-1, X_test.shape[-1]))
    results = results.reshape(X_test.shape[:-1])
    final_results = stats.mode(results, axis=1)
    return final_results.mode.flatten()

In [5]:
def predict_naive(model, X_test, scaler):
    X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    results = model.predict(X_test_scaled.reshape(-1, X_test.shape[-1]))

    return results

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def aggregate_metrics(y_true, y_pred):
    return accuracy_score(y_true, y_pred),\
        precision_score(y_true, y_pred, average='macro', zero_division=0.),\
        recall_score(y_true, y_pred, average='macro', zero_division=0.),\
        f1_score(y_true, y_pred, average='macro', zero_division=0.)

In [7]:
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score


def perform_experiment(train_folds, test_3sec_folds, test_30sec_folds, fold_idx,
                       type_return='report'):
    
    if type_return not in ['report', 'metrics']:
        raise ValueError("Invalid return_type. Must be either 'report' or 'metrics'.")
    
    X_train, y_train = train_folds[fold_idx]
    X_test, y_test = test_3sec_folds[fold_idx]
    X_test_30sec, y_test_30sec = test_30sec_folds[fold_idx]

    rand_perm = np.random.permutation(np.arange(X_test.shape[0]))
    X_test, y_test = X_test[rand_perm], y_test[rand_perm]
    X_test_30sec, y_test_30sec = X_test_30sec[rand_perm], y_test_30sec[rand_perm]
    
    y_train = np.repeat(y_train, X_train.shape[1])
    X_train = X_train.reshape(-1, X_train.shape[-1])

    rand_perm = np.random.permutation(np.arange(X_train.shape[0]))
    X_train, y_train = X_train[rand_perm], y_train[rand_perm]
    
    scalar = StandardScaler()
    X_train = scalar.fit_transform(X_train)

    # Change the model here to experiment with different models
    # model = SVC(kernel='rbf', decision_function_shape='ovo', C=1, probability=True, random_state=RANDOM_STATE)
    model = MLPClassifier(hidden_layer_sizes=(128, 128, 10), max_iter=1000, random_state=RANDOM_STATE)

    # model = RandomForestClassifier(max_features=20, max_depth=10)
    model.fit(X_train, y_train)

    if type_return == 'report':
        train_report = classification_report(y_train, model.predict(X_train))
        mode_report = classification_report(y_test, predict_and_mode(model, X_test, scalar))
        naive_report = classification_report(y_test_30sec, predict_naive(model, X_test_30sec, scalar)) 
        return train_report, mode_report, naive_report
    else:
        train_metrics = aggregate_metrics(y_train, model.predict(X_train))
        mode_metrics = aggregate_metrics(y_test, predict_and_mode(model, X_test, scalar))
        naive_metrics = aggregate_metrics(y_test_30sec, predict_naive(model, X_test_30sec, scalar))
        return train_metrics, mode_metrics, naive_metrics

In [8]:
def k_fold_experiment(train_folds, test_3sec_folds, test_30sec_folds):
    metrics = []
    for k in range(len(train_folds)):
        train_metrics, mode_metrics, naive_metrics = perform_experiment(train_folds, test_3sec_folds, test_30sec_folds, k, type_return='metrics')
        print(f"Fold {k+1}:")
        print(f"Train metrics: acc={train_metrics[0]:.3f}, precision={train_metrics[1]:.3f}, recall={train_metrics[2]:.3f}, f1={train_metrics[3]:.3f}")
        print(f"Mode Strategy metrics: acc={mode_metrics[0]:.3f}, precision={mode_metrics[1]:.3f}, recall={mode_metrics[2]:.3f}, f1={mode_metrics[3]:.3f}")
        print(f"Naive Strategy metrics: acc={naive_metrics[0]:.3f}, precision={naive_metrics[1]:.3f}, recall={naive_metrics[2]:.3f}, f1={naive_metrics[3]:.3f}")
        metrics.append((train_metrics, mode_metrics, naive_metrics))
    
    avg_accs = {
        "train": np.mean([m[0][0] for m in metrics], axis=0),
        "mode": np.mean([m[1][0] for m in metrics], axis=0),
        "naive": np.mean([m[2][0] for m in metrics], axis=0)
    }
    avg_precisions = {
        "train": np.mean([m[0][1] for m in metrics], axis=0),
        "mode": np.mean([m[1][1] for m in metrics], axis=0),
        "naive": np.mean([m[2][1] for m in metrics], axis=0)
    }
    avg_recalls = {
        "train": np.mean([m[0][2] for m in metrics], axis=0),
        "mode": np.mean([m[1][2] for m in metrics], axis=0),
        "naive": np.mean([m[2][2] for m in metrics], axis=0)
    }
    avg_f1s = {
        "train": np.mean([m[0][3] for m in metrics], axis=0),
        "mode": np.mean([m[1][3] for m in metrics], axis=0),
        "naive": np.mean([m[2][3] for m in metrics], axis=0)
    }
    
    print()
    print(f"Average train metrics: acc={avg_accs['train']:.3f}," +\
          f"precision={avg_precisions['train']:.3f}," +\
          f"recall={avg_recalls['train']:.3f}," +\
          f"f1={avg_f1s['train']:.3f}")
    print(f"Average Mode Strategy metrics: acc={avg_accs['mode']:.3f}," +\
          f"precision={avg_precisions['mode']:.3f}," +\
          f"recall={avg_recalls['mode']:.3f}," +\
          f"f1={avg_f1s['mode']:.3f}")
    print(f"Average Naive Strategy metrics: acc={avg_accs['naive']:.3f}," +\
          f"precision={avg_precisions['naive']:.3f}," +\
          f"recall={avg_recalls['naive']:.3f}," +\
          f"f1={avg_f1s['naive']:.3f}")

In [9]:
k_fold_experiment(train_folds, test_3sec_folds, test_30sec_folds)

Fold 1:
Train metrics: acc=0.999, precision=0.999, recall=0.999, f1=0.999
Mode Strategy metrics: acc=0.820, precision=0.793, recall=0.827, f1=0.799
Naive Strategy metrics: acc=0.790, precision=0.766, recall=0.784, f1=0.763
Fold 2:
Train metrics: acc=1.000, precision=1.000, recall=1.000, f1=1.000
Mode Strategy metrics: acc=0.780, precision=0.786, recall=0.782, f1=0.777
Naive Strategy metrics: acc=0.760, precision=0.732, recall=0.751, f1=0.734
Fold 3:
Train metrics: acc=0.999, precision=0.999, recall=0.999, f1=0.999
Mode Strategy metrics: acc=0.770, precision=0.773, recall=0.788, f1=0.764
Naive Strategy metrics: acc=0.780, precision=0.805, recall=0.790, f1=0.790
Fold 4:
Train metrics: acc=0.999, precision=0.999, recall=0.999, f1=0.999
Mode Strategy metrics: acc=0.790, precision=0.781, recall=0.769, f1=0.768
Naive Strategy metrics: acc=0.750, precision=0.730, recall=0.731, f1=0.723
Fold 5:
Train metrics: acc=1.000, precision=1.000, recall=1.000, f1=1.000
Mode Strategy metrics: acc=0.760, 